In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Dataset Description

Coronaviruses are a large family of viruses which may cause illness in animals or humans. In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19 - World Health Organization

The number of new cases are increasing day by day around the world. This dataset has information from the states and union territories of India at daily level.

State level data comes from Ministry of Health & Family Welfare,The Following data comes from "covid19indi" and th Information is highly authentic

# Importing the Required Libraries

In [ ]:
pip install pmdarima

In [ ]:
import pandas as pd
import plotly.express as ply
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels import tsa
import numpy as np
import statistics
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import folium
import os
from fbprophet import Prophet
import warnings
warnings.filterwarnings('ignore')
import plotly.graph_objects as go
pd.set_option('display.max_rows',20000, 'display.max_columns',100)
import datetime as datetime
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from math import  sqrt
from sklearn.metrics import mean_squared_error

In [ ]:
#pip install statsmodels --upgrade
!pip install statsmodels
!pip3 install --user scipy==1.2.0

In [ ]:
from pmdarima import auto_arima

# Loading The Data

In [ ]:
df=pd.read_csv("../input/covid19-in-india/covid_19_india.csv")

In [ ]:
df_corona_in_india = pd.read_csv("../input/covid19-in-india/covid_19_india.csv")


In [ ]:
df.head(5)

In [ ]:
df.tail(5)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

# Importing the metrics

In [ ]:
import datetime
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

In [ ]:
df['Date']=pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Indexing the Data

In [ ]:
df.set_index('Date',inplace=True)

# Resampling the Data by month and Date

In [ ]:
df['Confirmed'].resample(rule='M').mean().plot(figsize=(25,5))

In [ ]:
df['Confirmed'].resample(rule='Q').mean().plot(figsize=(25,5))


# Dropping the unnecessary columns

In [ ]:
df1=pd.DataFrame()
df1=df.drop(columns=['Sno','Time','ConfirmedIndianNational','State/UnionTerritory','ConfirmedForeignNational','Cured','Deaths'])

In [ ]:
df1.plot()

In [ ]:
df_day=df1.resample('D').sum()

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(df_day['Confirmed'])
result.plot()
plt.show()

# Checking the Stationarity in the Data

In [ ]:
def test_stationarity(timeseries):
    
    #Determing rolling statistics
    #rolmean = pd.Rolling.mean(timeseries, window=24)
    rolmean = timeseries.rolling(12).mean()
    #rolstd = pd.Rolling.std(timeseries, window=24)
    rolstd = timeseries.rolling(12).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))

    orig = plt.plot(timeseries, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()

    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
              dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)
    

In [ ]:
test_stationarity(df_day['Confirmed'])


# Applying log values to the Data

In [ ]:
import numpy as np
log_df = np.log(df_day)
plt.plot(log_df)

In [ ]:
df_day['diff1']=df_day['Confirmed']-df_day['Confirmed'].shift(1)
test_stationarity(df_day['diff1'].dropna())


# Plotting acf and pacf to find the stationarity in the data

In [ ]:
plot_acf(df_day['diff1'].dropna(),lags=15)
plt.show()

In [ ]:
plot_pacf(df_day['diff1'].dropna(),lags=15)
plt.show()

# Differencing the Data

In [ ]:
df_day['diff2']=df_day['Confirmed']-df_day['Confirmed'].shift(2)
test_stationarity(df_day['diff2'].dropna())


# Importing the Arima Model

In [ ]:
from statsmodels.tsa.arima.model import ARIMA


# Trying to find the best find arima model with the help of auto arima

In [ ]:
auto_arima_model=auto_arima(df_day['Confirmed'],trace=True,Supress_warnings=True)

In [ ]:
#Best Model 
arima_model_202 = ARIMA(df_day['Confirmed'].dropna(), order=(1,2,2)).fit()

# Plotting the arima model statistics

In [ ]:
arima_model_202.summary()

In [ ]:
start = df_day.index.get_loc('2020-10-6')
end = df_day.index.get_loc('2021-05-28')
print(start,'to',end)

# Predict the unseen Data using the best fit arima model we got using auto arima

In [ ]:
 predict_1= arima_model_202.predict(start=start+1 ,end=end+1, dynamic= False)  

# Forecasting the occurence of third wave in India with the help of Arima model

In [ ]:
#Trying to forecast future values
pred_future_10101=arima_model_202.predict(start=len(df_day)+1,end=len(df_day)+(180),dynamic=False)
print("The length of pred_future values :",len(pred_future_10101))

In [ ]:
plt.plot(df_day['Confirmed'],color='blue')
plt.plot(predict_1,color='yellow')
plt.plot(pred_future_10101,color='red')
plt.xticks(rotation=45)
plt.show()

# Data preprocessing

In [ ]:
df_corona_in_india.isnull().sum()


In [ ]:
df_corona_in_india['Deaths']=pd.to_numeric(df_corona_in_india['Deaths'])

# Data Visualization

# The first plot suggests that there is a constant increase in the total number of cases in India

In [ ]:
#Total cases of corona in India
df_corona_in_india['Total Cases'] = df_corona_in_india['Cured'] + df_corona_in_india['Deaths'] + df_corona_in_india['Confirmed']
#Active cases of corona in India
df_corona_in_india['Active Cases'] = df_corona_in_india['Total Cases'] - df_corona_in_india['Cured'] - df_corona_in_india['Deaths']
df_corona_in_india.head()

In [ ]:
df_corona_in_india.info()

In [ ]:
df_corona_in_india['Date'] =pd.to_datetime(df_corona_in_india.Date,dayfirst=True)


# The following plot gives information about the occurence total number of cases in India categorized by statewise

In [ ]:
#Total Cases in Indian States Datewise
corona_data = df_corona_in_india.groupby(['Date','State/UnionTerritory','Total Cases'])['Cured','Deaths','Active Cases'].sum().reset_index().sort_values('Total Cases',ascending = False)
fig = px.bar(corona_data, y='Total Cases', x='Date',hover_data =['State/UnionTerritory','Active Cases','Deaths','Cured'], color='Total Cases',barmode='group', height=600,color_continuous_scale=["blue","red"])
fig.update_layout(
    title='Indian States with Current Total Corona Cases')
fig.show()

# The following plot gives information about the number of persons thay were being cured Daily in India categorized by Datewise

In [ ]:
#Daily Cases in India Datewise
corona_data = df_corona_in_india.groupby(['Date'])['Cured'].sum().reset_index().sort_values('Cured',ascending = True)

fig = px.bar(corona_data, y='Cured', x='Date',hover_data =['Cured'], color='Cured', height=600,color_continuous_scale=["blue","red"])
fig.update_layout(
    title='Cured Cases in India Datewise')
fig.show()

# The following plot gives information about the number of deaths that were being occured Daily in India categorized by Datewise

In [ ]:
#Daily Cases in India Datewise
corona_data = df_corona_in_india.groupby(['Date'])['Total Cases'].sum().reset_index().sort_values('Total Cases',ascending = True)
corona_data['Daily Cases'] = corona_data['Total Cases'].sub(corona_data['Total Cases'].shift())
corona_data['Daily Cases'].iloc[0] = corona_data['Total Cases'].iloc[0]
corona_data['Daily Cases'] = corona_data['Daily Cases'].astype(int)
fig = px.bar(corona_data, y='Daily Cases', x='Date',hover_data =['Daily Cases'], color='Daily Cases', height=600,color_continuous_scale=["blue","red"])
fig.update_layout(
    title='Daily Cases in India Datewise')
fig.show()

In [ ]:
#Total Cases in India Datewise
corona_data = df_corona_in_india.groupby(['Date'])['Total Cases'].sum().reset_index().sort_values('Total Cases',ascending = True)
fig = px.bar(corona_data, y='Total Cases', x='Date',hover_data =['Total Cases'], color='Total Cases', height=600,color_continuous_scale=["blue","red"])
fig.update_layout(title='Total Cases in India Datewise')
fig.show()

# The following plot gives information about the growth in the percentage of occurence of new cases compared to the previous day

In [ ]:
# pct_change is used to obtain the percentage change in consecutive rows.
corona_data['Corona Growth Rate'] = corona_data['Total Cases'].pct_change().mul(100).round(2)
#Corona Growth Rate Comparison with Previous Day
fig = px.bar(corona_data, y='Corona Growth Rate', x='Date',hover_data =['Corona Growth Rate','Total Cases'], height=600,color_continuous_scale=["blue","red"])
fig.update_layout(title='Corona Growth Rate(in Percentage) Comparison with Previous Day')
fig.show()

# The following graph gives information about the mortality rate in India

In [ ]:
#Moratality Rate
corona_data = df_corona_in_india.groupby(['Date'])['Total Cases','Active Cases','Deaths'].sum().reset_index().sort_values('Date',ascending=False)
corona_data['Mortality Rate'] = ((corona_data['Deaths']/corona_data['Total Cases'])*100) #(Death/ total cases)*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=corona_data['Date'], y=corona_data['Mortality Rate'],mode='lines+markers',name='Cases',marker_color='red'))
fig.update_layout(title_text='COVID-19 Mortality Rate in INDIA',plot_bgcolor='rgb(225,230,255)' ,xaxis_title="Date",
    yaxis_title="Mortality Rate")
fig.show()

In [ ]:
#DAILY DEATHS IN INDIA
corona_data = df_corona_in_india.groupby(['Date'])['Deaths'].sum().reset_index().sort_values('Deaths',ascending = True)
corona_data['DAILY DEATHS'] = corona_data['Deaths'].sub(corona_data['Deaths'].shift())
corona_data['DAILY DEATHS'].iloc[0] = corona_data['Deaths'].iloc[0]
corona_data['DAILY DEATHS'] = corona_data['DAILY DEATHS'].astype(int)
fig = px.bar(corona_data, y='DAILY DEATHS', x='Date',hover_data =['DAILY DEATHS'], color='DAILY DEATHS', height=600,color_continuous_scale=["blue","red"])
fig.update_layout(
    title='DAILY DEATHS IN INDIA')
fig.show()

In [ ]:
#DAILY DEATHS IN INDIA
corona_data = df_corona_in_india.groupby(['Date'])['Deaths'].sum().reset_index().sort_values('Deaths',ascending = True)
fig = px.bar(corona_data, y='Deaths', x='Date',hover_data =['Deaths'], color='Deaths', height=600,color_continuous_scale=["blue","red"])
fig.update_layout(
    title='DEATHS IN INDIA')
fig.show()

#  The plot gives info about the Recovery rate In India

In [ ]:
#Recovery Rate
corona_data = df_corona_in_india.groupby(['Date'])['Total Cases','Active Cases','Cured'].sum().reset_index().sort_values('Date',ascending=False)
corona_data['Recovery Rate'] = ((corona_data['Cured']/corona_data['Total Cases'])*100)
fig = go.Figure()
corona_data = corona_data[3:]
fig.add_trace(go.Scatter(x=corona_data['Date'], y=corona_data['Recovery Rate'],
                    mode='lines+markers',name='Cases',marker_color='green'))


fig.update_layout(title_text='COVID-19 Recovery Rate in INDIA',plot_bgcolor='rgb(225,230,255)',xaxis_title="Date",
    yaxis_title="Recovery Rate")
fig.show()

#  The plot gives info about the Recovery rate In India DateWise

In [ ]:
#DAILY Recovery IN INDIA
corona_data = df_corona_in_india.groupby(['Date'])['Cured'].sum().reset_index().sort_values('Cured',ascending = True)
corona_data['DAILY RECOVERY'] = corona_data['Cured'].sub(corona_data['Cured'].shift())
corona_data['DAILY RECOVERY'].iloc[0] = corona_data['Cured'].iloc[0]
corona_data['DAILY RECOVERY'] = corona_data['DAILY RECOVERY'].astype(int)
fig = px.bar(corona_data, y='DAILY RECOVERY', x='Date',hover_data =['DAILY RECOVERY'], color='DAILY RECOVERY', height=600,color_continuous_scale=["blue","red"])
fig.update_layout(
    title='DAILY RECOVERY IN INDIA')
fig.show()

In [ ]:
#Total Cases in Indian States Datewise
corona_data = df_corona_in_india.groupby(['Date','State/UnionTerritory','Total Cases'])['Cured','Deaths','Active Cases'].sum().reset_index().sort_values('Total Cases',ascending = False)
fig = px.bar(corona_data, y='Total Cases', x='Date',hover_data =['State/UnionTerritory','Active Cases','Deaths','Cured'], color='Total Cases',barmode='group', height=600,color_continuous_scale=["blue","red"])
fig.update_layout(
    title='Indian States with Current Total Corona Cases')
fig.show()

#  The following plot gives information about the total cases and active cases and deaths and the amount of persons that were cured from coronavirus

In [ ]:
#Total Cases,Active Cases,Cured,Deaths from Corona Virus in India
corona_data = df_corona_in_india.groupby(['Date'])['Total Cases','Active Cases','Cured','Deaths'].sum().reset_index().sort_values('Date',ascending=False)
fig = go.Figure()
fig.add_trace(go.Scatter(x=corona_data['Date'], y=corona_data['Total Cases'],
                    mode='lines+markers',name='Total Cases'))
fig.add_trace(go.Scatter(x=corona_data['Date'], y=corona_data['Active Cases'], 
                mode='lines+markers',name='Active Cases'))
fig.add_trace(go.Scatter(x=corona_data['Date'], y=corona_data['Cured'], 
                mode='lines+markers',name='Cured'))
fig.add_trace(go.Scatter(x=corona_data['Date'], y=corona_data['Deaths'], 
                mode='lines+markers',name='Deaths'))
fig.update_layout(title_text='Curve Showing Different Cases from COVID-19 in India',plot_bgcolor='rgb(225,230,255)')
fig.show()

In [ ]:
df_corona_in_india['Total Cases'] = df_corona_in_india['Cured'] + df_corona_in_india['Deaths'] + df_corona_in_india['Confirmed']

# Forecasting the third wave  with the help of FB prophet

In [ ]:
#Forecasting of Total Cases for Next 30 Days
df = df_corona_in_india.groupby('Date')['Total Cases'].sum().reset_index()
# Assigining variables to dates and total cases(Target Class) 
df.columns = ['ds','y']
df['ds'] = pd.to_datetime(df['ds'])
# Prophet is a forcasting model made by Facebook
m = Prophet()
# Lets fit the model
m.fit(df)
# Getting the next 30 dates
future = m.make_future_dataframe(periods=90,include_history = False)
#Obtaining the forcast for the next 30 days
forecast = m.predict(future)
#Lets plot on the graph for a easy view and understanding
fig = go.Figure()
# yhat is the predicted value ds is the dates 
fig.add_trace(go.Scatter(x=forecast['ds'], y=forecast['yhat'],
                    mode='lines+markers',name='Cases',marker_color='Black'))
fig.update_layout(
    title='Forecasting of Total Cases in INDIA for Next 120 Days',xaxis_title="Date",
    yaxis_title="Count")
fig.show()
from fbprophet.diagnostics import cross_validation
# help(cross_validation)
df_cv = cross_validation(m, horizon='30 days', period='15 days', initial='1 days')
print(forecast)
m.plot(forecast)

# Conclusion

From the above Analysis we were able to state that there will be a huge increase in the number of cases in the coming 4 months by september.The number of people that will be affected and the mortality rate is going to increase gradually which may rose in to a dangerous situation in the coming 4 months so we need to take preventive measures and vaccination in order to prevent the occurence of third wave in India

# Kindly upvote if you liked my work